## Lecture III How is agricultural production increasing?



We&rsquo;ve seen that growth in food production is typically greater than
population growth.  But where is this growth coming from?  

Food (and crops in particular) are the classical example of 
production.  The &ldquo;classical&rdquo; economists in the 18th century, when
most income came from agriculture, thought there were three main
&ldquo;factors&rdquo; of production:

-   Land
-   Labor
-   Capital

Thus one might write the technical relationship between &ldquo;factors&rdquo; (or
inputs) and output as
$$
    \text{Crop output} = F(\mbox{Land},\mbox{Labor},\mbox{Capital}).
 $$



### Functional forms



We have good reason to think that $F$ displays constant returns to
scale; i.e., is homogeneous of degree one.  Write it as
$F(x_1,x_2,\dots,x_n)$ (thus abstracting from the classical factors
of production).  

Observationally, it&rsquo;s also often the case the cost share for
different factors of production remains constant, even when prices
change.  If we combine these facts (linearly homogeneous, constant
cost shares) with an assumption that farmers operating this
production function are profit maximizing price-takers and a
technical assumption that $F$ is continuously differentiable, then
one can prove that $F$ is &ldquo;Cobb-Douglas&rdquo;, or
$$
      F(x_1,x_2,\dots,x_n) = A\prod_{i=1}^nx_i^{\alpha_i},
  $$
where $\sum_{i=1}^n\alpha_i=1$.  This is a result first established
by the economist Paul Douglas and the mathematician Charles Cobb in 1928.

If we observe output at time $t$ for country $j$, say $y^j_t$ and inputs $x^j_{it}$, we can take the
logarithm of the Cobb-Douglas production function, obtaining
$$
     \log y^j_t = \log A^j_t + \sum_{i=1}^n\alpha^j_i\log(x^j_{it}).
  $$
Note that we&rsquo;ve allowed the cost-share parameters $\alpha$ to vary
across both inputs and also countries, but *not* over time.



### Total Factor Productivity



The term $A$ is sometimes called &ldquo;Total Factor Productivity&rdquo; (TFP),
because increases in $A$ increase productivity of all factors.  If
we take differences in log output over time we get
$$
      \Delta\log y^j_t = \Delta\log A^j_t + \sum_{i=1}^n\alpha^j_i\Delta\log(x^j_{it}).
   $$
Recall that changes in logs approximate percent changes or growth
rates, so we can use this equation to decompose output growth into
growth in input use and TFP growth.



### Data on Food Production



What’s happened to food production over recent decades?
See
[https://www.ers.usda.gov/data-products/international-agricultural-productivity/](https://www.ers.usda.gov/data-products/international-agricultural-productivity/).

Data on TFP, output, factor use, and factor shares can be found at
[https://docs.google.com/spreadsheets/d/1DLn9owcS7ggojJGWlI9vKSz0hqozn6cbcqNGWgzMZ8k](https://docs.google.com/spreadsheets/d/1DLn9owcS7ggojJGWlI9vKSz0hqozn6cbcqNGWgzMZ8k),
which is publicly readable.



### Reading Sheets



I&rsquo;ve written a python package `eep153_tools` which includes tools to
handle authentication as well as reading google sheets as pandas
DataFrames.  First we have to deal with authentication, by decrypting
credentials to access particular files (you should only have to do
this part once):



In [1]:
!pip install eep153_tools
!pip install python_gnupg

from eep153_tools.sheets import decrypt_credentials
decrypt_credentials('../students.json.gpg')

Input secret passphrase for ../students.json.gpg to create google drive credentials: noodle octopus


To check that this worked, the following gives a list of
emails for &ldquo;service<sub>accounts</sub>&rdquo; that now have credentials&#x2013;you can
then &ldquo;share&rdquo; google sheets with these.



In [2]:
!ls ~/.eep153.service_accounts/

students@eep153.iam.gserviceaccount.com


With those credentials established



In [3]:
from eep153_tools.sheets import read_sheets

#### Read a bunch of google worksheets into a dictionary of dataframes
data = read_sheets('https://docs.google.com/spreadsheets/d/1DLn9owcS7ggojJGWlI9vKSz0hqozn6cbcqNGWgzMZ8k',force_numeric=False)

data.keys()

Key available for students@eep153.iam.gserviceaccount.com.


dict_keys(['Explanation', 'Ag TFP', 'Output', 'Inputs', 'Factor Shares', 'Ag Land', 'Land Weights', 'Cropland', 'Irrig', 'Pasture', 'Labor', 'Livestock', 'Machinery', 'Fertilizer', 'Feed'])

This gives a dictionary of dataframes, but these dataframes need to be
tidied up some.  For example, look at Output:



In [4]:
data['Output'].head()

,Gross Agricultural Production,,,,,,,,,,...,,,,,,,,,,
0,"($1000, constant 2004-2006 prices)",,,,,,,,,,...,,,,,,,,,,
1,Order,FAO N,WDI Code,Country,Region,Sub-Region,Inc I,Inc II,Note,,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2,1,202,ZAF,South Africa,"AFRICA, Developed",South Africa,MI-U,MI-U-xC,,,...,-0.010,0.033,0.022,0.059,-0.023,-0.042,,,,
3,2,32,CMR,Cameroon,"AFRICA, SUB-SAHARA","SSA, Central",MI-L,MI-L,,,...,0.028,0.055,0.027,0.008,0.046,0.061,,,,
4,3,37,CAF,Central African Republic,"AFRICA, SUB-SAHARA","SSA, Central",LI,LI,,,...,0.050,0.016,0.011,-0.048,0.024,0.020,,,,


We really just want the columns that begin with years.  Also, there
are annoyances like commas in numbers that we need to deal with.  
Let&rsquo;s write a function to clean things up.



In [1]:
import pandas as pd

def get_international_ag_productivity_data(series):

    columns=series.iloc[1,:]  # Need to fix columns
    series = series.iloc[2:,:]
    series.columns = columns

    # Set index to country code
    series = series.set_index('WDI Code')

    series = series[['%d' % t for t in range(1961,2020)]]

    series = series.replace({',':''},regex=True) # Get rid of commas in number strings
    series = series.replace({'':'NaN'}) # Change empty cells to NaN strings
    series = series.apply(lambda x: pd.to_numeric(x,errors='ignore'))
    series = series.astype(float) # Convert to floats

    series = series.stack()
    
    series = series.loc[~series.index.duplicated(keep='first')] 

    series.index.names = ['WDI Code','Year']

    return series
   

Data = ['Output','Ag TFP','Ag Land','Irrig','Pasture','Labor','Livestock',
        'Machinery','Fertilizer','Feed']

D = {}
for key in Data:
    D[key] = get_international_ag_productivity_data(data[key])

df = pd.DataFrame(D)
df

### Visualizing data on ag production



Plot growth in output for all countries for 1961 on:



In [1]:
import cufflinks as cf
cf.go_offline()

df['Output'].unstack().T.iplot(title="Value of Agricultural Output",
                               yTitle='Thousands of 2005-06 Dollars',
                               xTitle='Year')

Compare world growth in outputs, inputs, and TFP:



In [1]:
import numpy as np

world = df.xs('WLD',level='WDI Code').replace(0,np.nan).dropna(how='any')

# Put in log differences
dworld = np.log(world).diff()
dworld['Inputs'] = dworld['Output'] - dworld['Ag TFP']

dworld.mean()

And a graph of growth rates:



In [1]:
dworld[['Output','Inputs','Ag TFP']].iplot(title="Growth rates of output, inputs, & TFP",
                                           xTitle="Year")

That&rsquo;s the overall picture for the world.  Now &ldquo;drill down&rdquo; and
consider just production in the US:



In [1]:
select = df.xs('USA',level='WDI Code').dropna(how='any')

dselect = np.log(select).diff()
dselect['Inputs'] = dselect['Output'] - dselect['Ag TFP']

dselect.mean()

And here a graph of growth in indices of inputs & outputs since 1961:



In [1]:
select = df.xs('USA',level='WDI Code').dropna(how='any')

select = select/select.loc['1961',:]
select.iplot()

Compare with India, which in recent years has had a level of
agricultural output close to that of the US:



In [1]:
select = df.xs('IND',level='WDI Code').dropna(how='any')

select = select/select.loc['1961',:]
select.iplot()

And now look at ratios of inputs & outputs in India to the same in the
US:



In [1]:
USA = df.xs('USA',level='WDI Code').dropna(how='any')

IND = df.xs('IND',level='WDI Code').dropna(how='any')

select = np.log(IND/USA
select.iplot(title='Log Indian inputs & output Relative to US')